### data analysis 기반 특징 별 가중치를 주어서 계산하여 추천 목록 만들기
- 기본적으로 '커피가 맛있다', '친절하다', '음료가 맛있다'가 공동 1-3위를 차지한다.
- 메뉴와 관련된 키워드를 제외하고 분석하면
- 북카페의 경우 그 다음으로는 '집중하기 좋다', '인테리어', '가성비', '대화하기 좋은', '좌석 편함'이 주로 비중을 차지하며
- 전체 조용한 카페와 북카페 데이터를 통합하였을 때 결과는 '인테리어', '청결', '대화하기 좋은', '집중하기 좋은', '가성비' 등이 주로 비중을 차지함.

- 커피가 맛있다, 음료가 맛있다, 디저트는 1점씩 부여
- 집중하기 좋다, 좌석 편함의 경우 5점 부여
- 가성비, 대화하기 좋은(조용하다는 것으로 생각), 청결의 경우 3점씩 부여
- 각 등장한 키워드 5개에 대한 결과값을 계산
- 평점이 4.3가 넘는 것에는 3점 부여

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('cafe_cau_for_database.csv')

In [3]:
data

,Unnamed: 0,place,rate,address,store_date,store_time,price,커피가맛있어요,음료가맛있어요,인테리어가멋져요,...,음식이맛있어요,매장이넓어요,매장이청결해요,주차하기편해요,빵이맛있어요,특별한메뉴가있어요,가성비가좋아요,재료가신선해요,좌석이편해요,차가맛있어요
0,0,토크넌센스,-1.00,서울 동작구 흑석로8길 7 Talk nonsense,"['월', '화', '수', '목', '금', '토', '일']","['10:00 - 22:00\n21:30 라스트오더', '10:00 - 22:00\...",4000.0,12,7,5,...,0,0,0,0,0,0,0,0,0,0
1,1,THE한강,4.63,서울 동작구 노량진로32길 29-6,"['월', '화', '수', '목', '금', '토', '일']","['09:00 - 22:00\n21:00 라스트오더', '09:00 - 22:00\...",3500.0,132,66,0,...,0,0,0,0,0,0,0,0,0,0
2,2,잔디속에있다고상상을해,-1.00,서울 동작구 상도로 265-30 2층,"['월', '화', '수', '목', '금', '토', '일']","['11:00 - 23:00\n22:30 라스트오더', '11:00 - 23:00\...",4500.0,22,13,16,...,0,0,0,0,0,0,0,0,0,0
3,3,뚜스뚜스 흑석역점,4.33,서울 동작구 현충로 75 원불교기념관 1층,['매일'],['07:00 - 23:30'],-1.0,0,0,93,...,402,274,104,100,0,0,0,0,0,0
4,4,브레드숨,4.41,서울 동작구 노량진로18길 37,"['월', '화', '수', '목', '금', '토', '일']","['10:30 - 19:30', '10:30 - 19:30', '10:30 - 19...",2300.0,0,0,0,...,0,0,15,0,205,47,21,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,152,마담콩cafe,4.36,서울 동작구 서달로12가길 8,[],[],-1.0,20,9,0,...,0,0,0,0,0,0,11,0,0,0
153,153,쉬어가는모퉁이커피한잔,4.31,서울 동작구 상도로55길 1,[],[],-1.0,7,3,0,...,0,0,0,0,0,0,3,0,0,0
154,154,보트커피,4.75,서울 동작구 장승배기로 56 지층 카페,"['월(8/29)', '화(8/30)', '수(8/31)', '목', '금', '토...","['휴무', '휴무', '휴무', '08:00 - 20:00', '08:00 - 2...",3000.0,37,24,0,...,0,0,0,0,0,13,20,0,0,0
155,155,비틀쥬스 중앙대학교병원점,-1.00,서울 동작구 흑석로 102 B115호,[],[],-1.0,1,12,0,...,0,0,0,0,0,6,0,0,0,0


In [4]:
# 인덱스 중복 column 드랍
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
data['score'] = 0 # 새로운 컬럼 생성

In [6]:
data

,place,rate,address,store_date,store_time,price,커피가맛있어요,음료가맛있어요,인테리어가멋져요,대화하기좋아요,...,매장이넓어요,매장이청결해요,주차하기편해요,빵이맛있어요,특별한메뉴가있어요,가성비가좋아요,재료가신선해요,좌석이편해요,차가맛있어요,score
0,토크넌센스,-1.00,서울 동작구 흑석로8길 7 Talk nonsense,"['월', '화', '수', '목', '금', '토', '일']","['10:00 - 22:00\n21:30 라스트오더', '10:00 - 22:00\...",4000.0,12,7,5,4,...,0,0,0,0,0,0,0,0,0,0
1,THE한강,4.63,서울 동작구 노량진로32길 29-6,"['월', '화', '수', '목', '금', '토', '일']","['09:00 - 22:00\n21:00 라스트오더', '09:00 - 22:00\...",3500.0,132,66,0,0,...,0,0,0,0,0,0,0,0,0,0
2,잔디속에있다고상상을해,-1.00,서울 동작구 상도로 265-30 2층,"['월', '화', '수', '목', '금', '토', '일']","['11:00 - 23:00\n22:30 라스트오더', '11:00 - 23:00\...",4500.0,22,13,16,0,...,0,0,0,0,0,0,0,0,0,0
3,뚜스뚜스 흑석역점,4.33,서울 동작구 현충로 75 원불교기념관 1층,['매일'],['07:00 - 23:30'],-1.0,0,0,93,0,...,274,104,100,0,0,0,0,0,0,0
4,브레드숨,4.41,서울 동작구 노량진로18길 37,"['월', '화', '수', '목', '금', '토', '일']","['10:30 - 19:30', '10:30 - 19:30', '10:30 - 19...",2300.0,0,0,0,0,...,0,15,0,205,47,21,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,마담콩cafe,4.36,서울 동작구 서달로12가길 8,[],[],-1.0,20,9,0,0,...,0,0,0,0,0,11,0,0,0,0
153,쉬어가는모퉁이커피한잔,4.31,서울 동작구 상도로55길 1,[],[],-1.0,7,3,0,0,...,0,0,0,0,0,3,0,0,0,0
154,보트커피,4.75,서울 동작구 장승배기로 56 지층 카페,"['월(8/29)', '화(8/30)', '수(8/31)', '목', '금', '토...","['휴무', '휴무', '휴무', '08:00 - 20:00', '08:00 - 2...",3000.0,37,24,0,0,...,0,0,0,0,13,20,0,0,0,0
155,비틀쥬스 중앙대학교병원점,-1.00,서울 동작구 흑석로 102 B115호,[],[],-1.0,1,12,0,0,...,0,0,0,0,6,0,0,0,0,0


In [7]:
data['score'] = np.where(data['rate'] > 4.3, data['score']+3, data['score'])
score_5_list = ['집중하기좋아요', '좌석이편해요']
score_3_list = ['가성비가좋아요', '대화하기좋아요', '매장이청결해요']
score_1_list = ['커피가맛있어요', '음료가맛있어요', '디저트가맛있어요']

for keyword in score_5_list:
    data['score'] = np.where(data[keyword] > 0, data['score']+5, data['score'])
    
for keyword in score_3_list:
    data['score'] = np.where(data[keyword] > 0, data['score']+3, data['score'])
    
for keyword in score_1_list:
    data['score'] = np.where(data[keyword] > 0, data['score']+1, data['score'])

In [8]:
data

,place,rate,address,store_date,store_time,price,커피가맛있어요,음료가맛있어요,인테리어가멋져요,대화하기좋아요,...,매장이넓어요,매장이청결해요,주차하기편해요,빵이맛있어요,특별한메뉴가있어요,가성비가좋아요,재료가신선해요,좌석이편해요,차가맛있어요,score
0,토크넌센스,-1.00,서울 동작구 흑석로8길 7 Talk nonsense,"['월', '화', '수', '목', '금', '토', '일']","['10:00 - 22:00\n21:30 라스트오더', '10:00 - 22:00\...",4000.0,12,7,5,4,...,0,0,0,0,0,0,0,0,0,6
1,THE한강,4.63,서울 동작구 노량진로32길 29-6,"['월', '화', '수', '목', '금', '토', '일']","['09:00 - 22:00\n21:00 라스트오더', '09:00 - 22:00\...",3500.0,132,66,0,0,...,0,0,0,0,0,0,0,0,0,5
2,잔디속에있다고상상을해,-1.00,서울 동작구 상도로 265-30 2층,"['월', '화', '수', '목', '금', '토', '일']","['11:00 - 23:00\n22:30 라스트오더', '11:00 - 23:00\...",4500.0,22,13,16,0,...,0,0,0,0,0,0,0,0,0,8
3,뚜스뚜스 흑석역점,4.33,서울 동작구 현충로 75 원불교기념관 1층,['매일'],['07:00 - 23:30'],-1.0,0,0,93,0,...,274,104,100,0,0,0,0,0,0,6
4,브레드숨,4.41,서울 동작구 노량진로18길 37,"['월', '화', '수', '목', '금', '토', '일']","['10:30 - 19:30', '10:30 - 19:30', '10:30 - 19...",2300.0,0,0,0,0,...,0,15,0,205,47,21,0,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,마담콩cafe,4.36,서울 동작구 서달로12가길 8,[],[],-1.0,20,9,0,0,...,0,0,0,0,0,11,0,0,0,9
153,쉬어가는모퉁이커피한잔,4.31,서울 동작구 상도로55길 1,[],[],-1.0,7,3,0,0,...,0,0,0,0,0,3,0,0,0,8
154,보트커피,4.75,서울 동작구 장승배기로 56 지층 카페,"['월(8/29)', '화(8/30)', '수(8/31)', '목', '금', '토...","['휴무', '휴무', '휴무', '08:00 - 20:00', '08:00 - 2...",3000.0,37,24,0,0,...,0,0,0,0,13,20,0,0,0,8
155,비틀쥬스 중앙대학교병원점,-1.00,서울 동작구 흑석로 102 B115호,[],[],-1.0,1,12,0,0,...,0,0,0,0,6,0,0,0,0,3


In [9]:
result = data[['place', 'rate', 'address', 'store_date', 'store_time', 'price', 'score']]
result

,place,rate,address,store_date,store_time,price,score
0,토크넌센스,-1.00,서울 동작구 흑석로8길 7 Talk nonsense,"['월', '화', '수', '목', '금', '토', '일']","['10:00 - 22:00\n21:30 라스트오더', '10:00 - 22:00\...",4000.0,6
1,THE한강,4.63,서울 동작구 노량진로32길 29-6,"['월', '화', '수', '목', '금', '토', '일']","['09:00 - 22:00\n21:00 라스트오더', '09:00 - 22:00\...",3500.0,5
2,잔디속에있다고상상을해,-1.00,서울 동작구 상도로 265-30 2층,"['월', '화', '수', '목', '금', '토', '일']","['11:00 - 23:00\n22:30 라스트오더', '11:00 - 23:00\...",4500.0,8
3,뚜스뚜스 흑석역점,4.33,서울 동작구 현충로 75 원불교기념관 1층,['매일'],['07:00 - 23:30'],-1.0,6
4,브레드숨,4.41,서울 동작구 노량진로18길 37,"['월', '화', '수', '목', '금', '토', '일']","['10:30 - 19:30', '10:30 - 19:30', '10:30 - 19...",2300.0,9
...,...,...,...,...,...,...,...
152,마담콩cafe,4.36,서울 동작구 서달로12가길 8,[],[],-1.0,9
153,쉬어가는모퉁이커피한잔,4.31,서울 동작구 상도로55길 1,[],[],-1.0,8
154,보트커피,4.75,서울 동작구 장승배기로 56 지층 카페,"['월(8/29)', '화(8/30)', '수(8/31)', '목', '금', '토...","['휴무', '휴무', '휴무', '08:00 - 20:00', '08:00 - 2...",3000.0,8
155,비틀쥬스 중앙대학교병원점,-1.00,서울 동작구 흑석로 102 B115호,[],[],-1.0,3


In [10]:
result = result.sort_values(by='score', ascending=False)

In [11]:
result

,place,rate,address,store_date,store_time,price,score
35,청맥살롱,4.81,서울 동작구 서달로 161-1 2층,"['월', '화', '수', '목', '금', '토', '일']","['11:00 - 21:00', '11:00 - 21:00', '11:00 - 21...",4000.0,17
59,블루포트 중앙대학교102관점,4.37,서울 동작구 흑석로 84 중앙대학교 102관,"['월', '화', '수', '목', '금', '토', '일']","['08:00 - 20:00', '08:00 - 20:00', '08:00 - 20...",1800.0,16
135,모코커피,4.42,서울 동작구 만양로 19 단지내상가 1층 107호,['매일'],['10:00 - 20:00'],3500.0,14
126,중앙대310관 카페,4.40,서울 동작구 흑석로 84,[],[],1800.0,14
150,카페아드리아,4.31,서울 동작구 서달로 142,[],[],-1.0,13
...,...,...,...,...,...,...,...
100,스노우플라워,4.29,서울 동작구 서달로 167,[],[],7000.0,3
155,비틀쥬스 중앙대학교병원점,-1.00,서울 동작구 흑석로 102 B115호,[],[],-1.0,3
14,브라운필 빵공장,-1.00,서울 동작구 흑석로 11 1층,['매일'],['07:30 - 23:00'],3000.0,2
13,액션커피,-1.00,서울 동작구 서달로15길 23 104호 액션커피,"['월', '화', '수', '목', '금', '토', '일']","['11:00 - 21:00\n20:50 라스트오더', '11:00 - 21:00\...",4500.0,2


In [12]:
result = result.iloc[:20, :]

In [13]:
result

,place,rate,address,store_date,store_time,price,score
35,청맥살롱,4.81,서울 동작구 서달로 161-1 2층,"['월', '화', '수', '목', '금', '토', '일']","['11:00 - 21:00', '11:00 - 21:00', '11:00 - 21...",4000.0,17
59,블루포트 중앙대학교102관점,4.37,서울 동작구 흑석로 84 중앙대학교 102관,"['월', '화', '수', '목', '금', '토', '일']","['08:00 - 20:00', '08:00 - 20:00', '08:00 - 20...",1800.0,16
135,모코커피,4.42,서울 동작구 만양로 19 단지내상가 1층 107호,['매일'],['10:00 - 20:00'],3500.0,14
126,중앙대310관 카페,4.40,서울 동작구 흑석로 84,[],[],1800.0,14
150,카페아드리아,4.31,서울 동작구 서달로 142,[],[],-1.0,13
37,LITITCOFFEE,4.55,서울 동작구 상도로47바길 48 1층 릿잇커피,['매일'],['11:00 - 새벽 01:00'],3800.0,13
81,카페필림,4.49,서울 동작구 흑석로 8-1,"['월', '화', '수', '목', '금', '토', '일']","['09:00 - 23:00', '09:00 - 23:00', '09:00 - 23...",3000.0,13
55,흑석커피,4.49,서울 동작구 서달로14가길 5 1층,"['월', '화', '수', '목', '금', '토', '일']","['10:00 - 22:00', '10:00 - 22:00', '10:00 - 22...",1800.0,13
26,커피나무 랩,4.48,서울 동작구 상도로53길 65 커피나무빌딩 1층,"['월', '화', '수', '목', '금', '토', '일']","['08:00 - 23:00', '08:00 - 23:00', '08:00 - 23...",4300.0,13
76,탐앤탐스 중앙대점,4.34,서울 동작구 흑석로 105 2층,['매일'],['08:00 - 23:00\n22:30 라스트오더'],-1.0,12


In [14]:
result.index = np.arange(0, len(result), 1)
result.drop(['score'], axis=1, inplace=True)

/Users/byeorikim/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
result

,place,rate,address,store_date,store_time,price
0,청맥살롱,4.81,서울 동작구 서달로 161-1 2층,"['월', '화', '수', '목', '금', '토', '일']","['11:00 - 21:00', '11:00 - 21:00', '11:00 - 21...",4000.0
1,블루포트 중앙대학교102관점,4.37,서울 동작구 흑석로 84 중앙대학교 102관,"['월', '화', '수', '목', '금', '토', '일']","['08:00 - 20:00', '08:00 - 20:00', '08:00 - 20...",1800.0
2,모코커피,4.42,서울 동작구 만양로 19 단지내상가 1층 107호,['매일'],['10:00 - 20:00'],3500.0
3,중앙대310관 카페,4.40,서울 동작구 흑석로 84,[],[],1800.0
4,카페아드리아,4.31,서울 동작구 서달로 142,[],[],-1.0
5,LITITCOFFEE,4.55,서울 동작구 상도로47바길 48 1층 릿잇커피,['매일'],['11:00 - 새벽 01:00'],3800.0
6,카페필림,4.49,서울 동작구 흑석로 8-1,"['월', '화', '수', '목', '금', '토', '일']","['09:00 - 23:00', '09:00 - 23:00', '09:00 - 23...",3000.0
7,흑석커피,4.49,서울 동작구 서달로14가길 5 1층,"['월', '화', '수', '목', '금', '토', '일']","['10:00 - 22:00', '10:00 - 22:00', '10:00 - 22...",1800.0
8,커피나무 랩,4.48,서울 동작구 상도로53길 65 커피나무빌딩 1층,"['월', '화', '수', '목', '금', '토', '일']","['08:00 - 23:00', '08:00 - 23:00', '08:00 - 23...",4300.0
9,탐앤탐스 중앙대점,4.34,서울 동작구 흑석로 105 2층,['매일'],['08:00 - 23:00\n22:30 라스트오더'],-1.0


In [16]:
result.to_csv('recommend_cafe_for_study.csv')